## run LLMStudio core capabilities
* python 3.12
* scope of interess for the plugin: `llmstudio/engine`

reproduce:
1. clone llmstudio. navigate to project folder
1. `pip install -e .` "-e if you want to edit the source"
1. add your "OPENAI_API_KEY" to your environment or to a .env file
1. run the code below

In [1]:
from llmstudio.engine.providers.azure import AzureProvider
from llmstudio.engine.providers.openai import OpenAIProvider


def LLM(config):
    provider_map =dict(
    openai=OpenAIProvider,
    azure=AzureProvider)

    provider = provider_map.get(config.id)
    if provider:
        return provider(config=config)
    raise NotImplementedError(f"Provider not found: {config.id}")

/Users/diogo/fun/LLMstudio/venv/lib/python3.12/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [2]:
from llmstudio.engine import ModelConfig, ProviderConfig
provider_config = ProviderConfig(
        id="openai",
        name="OpenAI",
        chat=True,
        embed=False,
        keys=["OPENAI_API_KEY"],
        models={
            "gpt-3.5-turbo": ModelConfig(
                mode="chat",
                max_tokens=2048,
                input_token_cost=0.0000015,
                output_token_cost=0.000002,
            )
        },
    )

llm = LLM(config=provider_config)
llm

In [3]:
import os
import json
from dotenv import load_dotenv
load_dotenv()


chat_request = dict(
    chat_input="Hello?"

    ,api_key=os.environ["OPENAI_API_KEY"]
    ,model="gpt-3.5-turbo"
    ,parameters={"temperature": 0, "max_tokens":100} 
    ,is_stream = False
    ,has_end_token = False
    ,functions = None
    ,session_id = "this-is-a-test"
    ,retries=0)

result = await llm.chat(chat_request)
result = json.loads(result.body)

In [4]:
result["chat_output"]

'Hello! How can I assist you today?'

In [5]:
result.get("metrics")

{'input_tokens': 2,
 'output_tokens': 9,
 'total_tokens': 11,
 'cost_usd': 2.1000000000000002e-05,
 'latency_s': 1.032789945602417,
 'time_to_first_token_s': 0.9374516010284424,
 'inter_token_latency_s': 0.008394217491149903,
 'tokens_per_second': 10.650762090430499}

## running proxy server

In [11]:
from llmstudio.cli import start_proxy_server

proxy = start_proxy_server(host="0.0.0.0", port="8001")

Running LLMstudio Engine on http://0.0.0.0:8001 


In [2]:
proxy.kill()
